In [57]:
blendsql = """
{{
    LLMQA(
        'Who is the author of the book Riflemans Creed?',
        (SELECT title, content FROM documents WHERE documents MATCH 'Riflemans Creed'),
        options='w::notable commanding officers'    )
}}
"""

In [58]:
import re
from blendsql.grammars._peg_grammar import grammar

In [59]:
for parse_results, _, _ in grammar.scanString(blendsql):
    parsed_results_dict = parse_results.as_dict()
    if len(parsed_results_dict["args"]) > 0:
        blendsql = re.sub(re.escape(parsed_results_dict["args"][0]), re.sub(r'(\'|\")', '', parsed_results_dict["args"][0]), blendsql)

In [60]:
re.sub(r'(\'|\")', '', parsed_results_dict["args"][0])

'Who is the author of the book Riflemans Creed?'

In [61]:
parsed_results_dict["args"][0]

'Who is the author of the book Riflemans Creed?'

In [62]:
print(blendsql)


{{
    LLMQA(
        'Who is the author of the book Riflemans Creed?',
        (SELECT title, content FROM documents WHERE documents MATCH 'Riflemans Creed'),
        options='w::notable commanding officers'    )
}}



In [18]:
for match in re.finditer(r'(?<=MATCH )(\'|\").*(\'|\")', blendsql):
    fts5_q = match.group()
    blendsql = re.sub(r'MATCH {}'.format(fts5_q), f'MATCH \'{re.sub(r"-", " ", fts5_q[1:-1])}\'', blendsql)
    blendsql = re.sub(r'MATCH {}'.format(fts5_q), f'MATCH \'{re.sub(r"[^0-9a-zA-Z ]", "", fts5_q[1:-1])}\'', blendsql)


In [19]:
blendsql

'\nSELECT "battalion name" FROM w WHERE "location ( s )" LIKE \'%Guantanamo Bay%\' AND "notable commanding officers" = {{\n    LLMQA(\n        \'Who is the author of the book Rifleman\'s Creed?\',\n        (SELECT title, content FROM documents WHERE documents MATCH \'Riflemans Creed\'),\n        options=\'w::notable commanding officers\'    )\n}}\n'